In [1]:
import pandas as pd

df = pd.read_csv(
    "BTCUSD_Daily_OHLC.csv",    
    usecols=['timestamp', 'open', 'high', 'low', 'close', 'volume','trades'],
    dtype={'open': float, 'high': float, 'low': float, 'close': float, 'volume': float, 'trades': int}
)
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
df.set_index('timestamp', inplace=True)
ohlcv_df = df.copy()


In [ ]:
from candlestick import candlestick
def candle_pattern(function :str, df : pd.DataFrame) -> pd.Series:
    target=function
    function = getattr(candlestick, function)
    trend = function(df,ohlc=['open', 'high', 'low', 'close'],target=target)
    trend = trend[target].astype('bool').fillna(False).astype(int)
    return trend
def candle_trend_timestamp(weight_pattern,df)-> pd.Series:
    trends = pd.DataFrame()
    for pattern, weight in weight_pattern.items():
        trend = candle_pattern(pattern,df)
        if not trend.empty:
            trends[pattern] = trend * weight
    trend_bycandels = trends.sum(axis=1).reset_index(drop=True)
    return trend_bycandels
functions = dir(candlestick)
functions = [element for element in functions if not str(element).startswith('__')]
functions.remove("re")
weight_pattern = dict.fromkeys(functions)
weights = [1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2]

if len(weights) == len(weight_pattern):
    keys = list(weight_pattern.keys())  # Obtener las claves como lista
    for i, key in enumerate(keys):
        weight_pattern[key] = weights[i]
trend_bycandels = candle_trend_timestamp(weight_pattern,df)
print(trend_bycandels)



In [ ]:

# import plotly.graph_objects as go

# fig = go.Figure(data=[
#     go.Candlestick(
#         x=ohlcv_df.index,
#         open=ohlcv_df['open'],
#         high=ohlcv_df['high'],
#         low=ohlcv_df['low'],
#         close=ohlcv_df['close'],
#         name="Velas"
#     )
# ])

# fig.update_layout(
#     title="Gráfico de Velas Interactivo",
#     xaxis_title="Tiempo",
#     yaxis_title="Precio",
#     yaxis=dict(
#         fixedrange=False  # ⬅️ Esto permite zoom dinámico vertical
#     ),
#     dragmode='zoom',  # Habilita zoom rectangular
#     xaxis_rangeslider_visible=False,
#     template="plotly_dark",  # Estilo oscuro como TradingView
#     autosize=True,
#     height=700
# )

# fig.show()


In [ ]:
import pandas_ta  as ta
from indicators import IndicatorsWeather

indicator = ta.AnalysisIndicators(ohlcv_df)
indicators_list = indicator.indicators(as_list=True)
#indicators = {}
#for indicator in indicators_list:
#    indicators[indicator] = {}
#weights = [1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2]

strategy = IndicatorsWeather(ohlcv_df)
indicators = {
    ("sma","sma") : [{"length" :50},{"length":200}],
    "rsi": {"length" :14},
}
calculated_indicators = strategy.calculate_indicators(indicators)
weights = [3,7]
types = ["cruce","umbral"]
parameters = [{"fast":"sma50","slow":"sma200"},{"lower":30,"upper":70,"activator":calculated_indicators["rsi"]},{}]


signals, debug = strategy.generate_signals(calculated_indicators,weights,types,parameters)
print(debug[(debug["signal_sma"] != 0) & (debug["signal_rsi"] != 0)])

                  sma50        sma200  signal_sma     RSI_14  signal_rsi  \
timestamp                                                                  
2015-10-29    248.69318    248.225356         1.0  80.313999          -1   
2019-04-23   4488.54200   4477.598500         1.0  75.003797          -1   
2023-10-30  28362.78000  28247.445000         1.0  81.888991          -1   

            SIGNALS  
timestamp            
2015-10-29     -4.0  
2019-04-23     -4.0  
2023-10-30     -4.0  


In [ ]:
import pandas_ta  as ta
import inspect
def visible_functions(module):
    return [name for name, obj in inspect.getmembers(module) if inspect.isfunction(obj) and obj.__module__ == module.__name__]
trend = visible_functions(ta.overlap)
print(trend)
